In [1]:
import json
import numpy as np
import pandas as pd
import requests
import joblib
from pathlib import Path

MODEL_PATH = Path("models") / "lr_near_envelope_q95.joblib"
model = joblib.load(MODEL_PATH)
print("Loaded model:", MODEL_PATH)


Loaded model: models\lr_near_envelope_q95.joblib


In [5]:
KSC_LAT, KSC_LON = 28.5, -80.6
DRONE_LAT, DRONE_LON = 31.0, -79.0

def month_to_season(m):
    if m in [12, 1, 2]: return "DJF"
    if m in [3, 4, 5]: return "MAM"
    if m in [6, 7, 8]: return "JJA"
    return "SON"

def fetch_ecmwf_atmos(lat, lon, start_date, end_date, hourly_vars):
    """ECMWF atmospheric forecast (Open-Meteo ECMWF endpoint)."""
    url = "https://api.open-meteo.com/v1/ecmwf"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ",".join(hourly_vars),
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "UTC",
    }
    r = requests.get(url, params=params, timeout=60)
    if not r.ok:
        raise ValueError(f"Open-Meteo ECMWF error {r.status_code}: {r.text}")
    data = r.json()
    hourly = data.get("hourly", {})
    t = pd.to_datetime(hourly.get("time", []), utc=True)
    df = pd.DataFrame({"time": t})
    for v in hourly_vars:
        df[v] = hourly.get(v, np.nan)
    return df

def fetch_marine_waves(lat, lon, start_date, end_date, hourly_vars):
    """Marine wave forecast (Open-Meteo Marine endpoint)."""
    url = "https://marine-api.open-meteo.com/v1/marine"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ",".join(hourly_vars),
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "UTC",
    }
    r = requests.get(url, params=params, timeout=60)
    if not r.ok:
        raise ValueError(f"Open-Meteo Marine error {r.status_code}: {r.text}")
    data = r.json()
    hourly = data.get("hourly", {})
    t = pd.to_datetime(hourly.get("time", []), utc=True)
    df = pd.DataFrame({"time": t})
    for v in hourly_vars:
        df[v] = hourly.get(v, np.nan)
    return df


def build_feature_row(ksc_df, wave_df, launch_dt_utc, rocket_name="Falcon 9"):
    launch_dt_utc = pd.to_datetime(launch_dt_utc, utc=True)
    launch_hour_dt = launch_dt_utc.floor("h")

    k = ksc_df.loc[ksc_df["time"] == launch_hour_dt]
    w = wave_df.loc[wave_df["time"] == launch_hour_dt]
    if k.empty or w.empty:
        raise ValueError("Requested hour not found in forecast range.")

    k = k.iloc[0]
    w = w.iloc[0]

    # Map forecast → features
    wind_speed_10m = float(k["wind_speed_10m"])  # m/s
    t2m_C = float(k["temperature_2m"])          # °C
    tcc_frac = float(k["cloud_cover"]) / 100.0   # 0–1
    tp_mmhr = float(k["precipitation"])         # mm/hr (API convention)
    cp_mmhr = 0.0                               # not available here; set to 0
    msl = float(k["pressure_msl"]) * 100.0      # hPa → Pa
    swh = float(w["wave_height"])               # meters

    launch_hour = launch_hour_dt.hour
    launch_month = launch_hour_dt.month
    season = month_to_season(launch_month)

    X = pd.DataFrame([{
        "wind_speed_10m": wind_speed_10m,
        "t2m_C": t2m_C,
        "tcc_frac": tcc_frac,
        "tp_mmhr": tp_mmhr,
        "cp_mmhr": cp_mmhr,
        "msl": msl,
        "swh": swh,
        "launch_hour": launch_hour,
        "launch_month": launch_month,
        "season": season,
        "rocket_name": rocket_name,
    }])
    return X, launch_hour_dt

def predict_launch_weather_confidence(launch_dt_utc, rocket_name="Falcon 9"):
    launch_dt, messages = validate_and_snap_launch_datetime(
        launch_dt_utc,
        max_horizon_days=16,
        min_lead_hours=0,
    )

    start_date = launch_dt.floor("D").strftime("%Y-%m-%d")
    end_date = (launch_dt.floor("D") + pd.Timedelta(days=2)).strftime("%Y-%m-%d")

    ksc_vars = ["temperature_2m", "precipitation", "pressure_msl", "cloud_cover", "wind_speed_10m"]
    wave_vars = ["wave_height"]

    ksc_fcst = fetch_ecmwf_atmos(KSC_LAT, KSC_LON, start_date, end_date, ksc_vars)
    wave_fcst = fetch_marine_waves(DRONE_LAT, DRONE_LON, start_date, end_date, wave_vars)
    print("KSC columns:", list(ksc_fcst.columns))
    print("Wave columns:", list(wave_fcst.columns))

    X_new, used_hour = build_feature_row(ksc_fcst, wave_fcst, launch_dt, rocket_name=rocket_name)

    p_near = float(model.predict_proba(X_new)[0, 1])
    confidence_ok = 1.0 - p_near

    return {
        "requested_datetime_utc": str(pd.to_datetime(launch_dt_utc, utc=True)),
        "used_forecast_hour_utc": str(used_hour),
        "p_near_envelope": p_near,
        "confidence_weather_ok": confidence_ok,
        "messages": messages,
        "features_used": X_new.iloc[0].to_dict(),
    }


def validate_and_snap_launch_datetime(
    launch_dt_utc,
    max_horizon_days=16,
    min_lead_hours=0,
):
    messages = []

    requested_dt = pd.to_datetime(launch_dt_utc, utc=True)
    snapped_dt = requested_dt

    # Robust UTC "now"
    now = pd.Timestamp.now(tz="UTC").floor("h")

    forecast_start = now + pd.Timedelta(hours=min_lead_hours)
    forecast_end = now.floor("D") + pd.Timedelta(days=max_horizon_days)

    if snapped_dt < forecast_start:
        messages.append(
            f"Requested datetime {requested_dt} is in the past or too close to now. "
            f"Snapped to next available forecast hour {forecast_start}."
        )
        snapped_dt = forecast_start

    if snapped_dt > forecast_end:
        messages.append(
            f"Requested datetime {requested_dt} exceeds ECMWF forecast horizon "
            f"(~{max_horizon_days} days). Snapped to last available forecast hour {forecast_end}."
        )
        snapped_dt = forecast_end

    snapped_dt_hour = snapped_dt.floor("h")
    if snapped_dt_hour != snapped_dt:
        messages.append(f"Datetime snapped to hourly resolution: {snapped_dt_hour}.")
        snapped_dt = snapped_dt_hour

    if not messages:
        messages.append("Requested datetime was within forecast range; no snapping applied.")

    return snapped_dt, messages

def pretty_print_confidence(result):
    conf = 100 * result["confidence_weather_ok"]
    pnear = 100 * result["p_near_envelope"]
    print(f"Launch time (UTC): {result['used_forecast_hour_utc']}")
    print(f"Weather confidence (within historical envelope): {conf:.1f}%")
    print(f"Near-envelope risk: {pnear:.1f}%")
    if result.get("messages"):
        print("Notes:")
        for m in result["messages"]:
            print(" -", m)
def risk_bucket(p_near):
    if p_near < 0.15:
        return "GREEN (low near-envelope risk)"
    elif p_near < 0.35:
        return "YELLOW (moderate near-envelope risk)"
    else:
        return "RED (high near-envelope risk)"





In [6]:

result = predict_launch_weather_confidence("2025-12-20 23:00:00+00:00", rocket_name="Falcon 9")
pretty_print_confidence(result)
print("Risk category:", risk_bucket(result["p_near_envelope"]))

KSC columns: ['time', 'temperature_2m', 'precipitation', 'pressure_msl', 'cloud_cover', 'wind_speed_10m']
Wave columns: ['time', 'wave_height']
Launch time (UTC): 2025-12-20 23:00:00+00:00
Weather confidence (within historical envelope): 80.4%
Near-envelope risk: 19.6%
Notes:
 - Requested datetime was within forecast range; no snapping applied.
Risk category: YELLOW (moderate near-envelope risk)
